In [28]:
import gym
import time
import math
import matplotlib.pyplot as plt
import seaborn as sns
import numpy
import tensorflow as tf

In [29]:
print(tf.__version__)

2.4.1


In [45]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.initializers import RandomUniform 
from tensorflow.keras.optimizers import Adam

In [21]:
env = gym.make('CartPole-v1')

Set up the hyperparameters

In [22]:
n_episodes = 20000
batch_size = 32
learning_rate = 0.001

In [38]:
env.action_space.n

2

In [23]:
tf.keras.Dense()

AttributeError: module 'tensorflow.keras' has no attribute 'Dense'

In [48]:
class DQNAgent():
    
    def __init__(self):

        # Main model
        self.model = self.create_model()

        # Target network
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())

        # An array with last n steps for training
        self.replay_memory = deque(maxlen=REPLAY_MEMORY_SIZE)

        # Custom tensorboard object
        self.tensorboard = ModifiedTensorBoard(log_dir="logs/{}-{}".format(MODEL_NAME, int(time.time())))

        # Used to count when to update target network with main network's weights
        self.target_update_counter = 0
        
        
    def train_model(self):
        model = Sequential()
        model.add(Dense(100, activation='relu', kernel_initializer='he_normal', input_shape=env.observation_space.shape))
        model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
        model.add(Dense(env.action_space.n, activation=None, kernel_initializer=RandomUniform(minval=-0.03, maxval=0.03),
                        bias_initializer=tf.keras.initializers.Constant(-0.2)))
        model.compile(loss="mse", optimizer=Adam(learning_rate), metrics=['accuracy'])
        return model

    def update_replay_memory(self, transition):
        self.replay_memory.append(transition)
    
    def get_qs(self, state):
        return self.model.predict(np.array(state).reshape(-1, *state.shape)/255)[0]

In [49]:
from keras.callbacks import TensorBoard

#...

# Own Tensorboard class
class ModifiedTensorBoard(TensorBoard):

    # Overriding init to set initial step and writer (we want one log file for all .fit() calls)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.step = 1
        self.writer = tf.summary.FileWriter(self.log_dir)

    # Overriding this method to stop creating default log writer
    def set_model(self, model):
        pass

    # Overrided, saves logs with our step number
    # (otherwise every .fit() will start writing from 0th step)
    def on_epoch_end(self, epoch, logs=None):
        self.update_stats(**logs)

    # Overrided
    # We train for one batch only, no need to save anything at epoch end
    def on_batch_end(self, batch, logs=None):
        pass

    # Overrided, so won't close writer
    def on_train_end(self, _):
        pass

    # Custom method for saving own metrics
    # Creates writer, writes custom metrics and closes writer
    def update_stats(self, **stats):
        self._write_logs(stats, self.step)

In [46]:
model = Sequential()
model.add(Dense(100, activation='relu', kernel_initializer='he_normal', input_shape=env.observation_space.shape))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(env.action_space.n, activation=None, kernel_initializer=RandomUniform(minval=-0.03, maxval=0.03),
               bias_initializer=tf.keras.initializers.Constant(-0.2)))
model.compile(loss="mse", optimizer=Adam(learning_rate), metrics=['accuracy'])